In [ ]:
import pandas as pd

clean = pd.read_json('---')
clean.columns = ['anchor', 'positive', 'original']
entries = pd.DataFrame(columns=['anchor', 'positive', 'negative'])

In [ ]:
import requests

def get_all(url='http://localhost:5173/api/data/all'):
    response = requests.get(url)
    data = response.json()
    return data

def get_positive(anchor):
    url = f"http://localhost:5173/api/data/{anchor}/positive"
    return get_all(url)

def get_rand_negative(anchor):
    url = f"http://localhost:5173/api/data/{anchor}/negative"
    return get_all(url)
    
def get_adjacent(anchor):
    url = f"http://localhost:5173/api/data/{anchor}/adjacent"
    return get_all(url)

In [ ]:
clean.describe()

In [ ]:
queries = pd.DataFrame(get_all(), columns=['anchor'])
queries.drop(queries[queries['anchor'].str.contains('/')].index, inplace=True)
queries

In [ ]:
for row in clean.iterrows():
    index, data = row
    
    print(data['anchor'])
    try:
        adj = pd.DataFrame(get_adjacent(data['original'])).dropna().sample().iloc[0]
        positive = pd.DataFrame(get_positive(adj[0])).dropna().sample().iloc[0].loc['document']
        entries.loc[len(entries)] = [data['anchor'], data['positive'], positive]
    except:
        print('error')
        continue

entries

In [ ]:
entries

In [ ]:
entries.to_json('---', orient='records')

In [ ]:
import pandas as pd

spoilers = pd.read_json("---")
spoilers.columns = ["text", "spoiler", "label"]

In [ ]:
spoilers.describe()

In [ ]:
spoilers[spoilers.duplicated()]

In [ ]:
spoilers.drop_duplicates(inplace=True)
spoilers.describe()

In [ ]:
spoilers.sort_values(by='label', inplace=True, ascending=False)
spoilers.reset_index(drop=True, inplace=True)
spoilers

In [ ]:
spoilers[spoilers['text'].str.contains("^\\s.{,39}:\\s")]['text']

In [ ]:
import re

spoilers['text'] = spoilers['text'].apply(lambda x: re.sub(r'^.{,40}:\s', '', x))
spoilers

In [ ]:
# replace text that starts with a colon and a space with an empty string
spoilers['text'] = spoilers['text'].str.replace('\n', '')
spoilers['text'] = spoilers['text'].str.replace('\\s+', ' ', regex=True)
spoilers['text'] = spoilers['text'].str.replace('"', '')
spoilers

In [ ]:
spoilers['text_length'] = spoilers['text'].apply(len)
spoilers.sort_values(by='text_length', inplace=True, ascending=True)
spoilers['text_length'].describe()

In [ ]:
# change column types
spoilers['text'] = spoilers['text'].astype(str)
spoilers['spoiler'] = spoilers['spoiler'].astype(str)
spoilers['label'] = spoilers['label'].astype(bool)
spoilers.dtypes

In [ ]:
# show all rows with text length less than 60 and label 1
short_spoiler =  spoilers[(spoilers['text_length'] < 35) & (spoilers['label'] == 1)]
short_spoiler

In [ ]:
short_no_spoiler = spoilers[(spoilers['text_length'] < 100) & (spoilers['label'] == 0)]
short_no_spoiler

In [ ]:
long_spoiler = spoilers[(spoilers['text_length'] > 150) & (spoilers['label'] == 1)]
long_spoiler

In [ ]:
spoilers['spoiler'] = spoilers['spoiler'].str.replace('\n', '')
spoilers['spoiler'] = spoilers['spoiler'].str.replace('\\s+', ' ', regex=True)
spoilers['spoiler'] = spoilers['spoiler'].str.replace('"', '')
spoilers

In [ ]:
# calculate the ratio of spoiler length to text length
spoilers['ratio'] = spoilers['spoiler'].apply(len) / spoilers['text_length']
spoilers.sort_values(by='ratio', inplace=True, ascending=False)
spoilers

In [ ]:
spoilers[spoilers['label']==True]['ratio'].describe()

In [ ]:
# drop text_length and label the columns
spoilers.drop(columns=['text_length', 'label'], inplace=True)
spoilers.columns = ['text', 'spoiler', 'spoiler/text']
spoilers

In [ ]:
spoilers.drop_duplicates(inplace=True)
spoilers.info()

In [ ]:
spoilers.to_json('---', orient='records')

In [ ]:
params = {'id1': 'Koyomi Araragi', 'id2': 'Hitagi Senjougahara'}
response = requests.post('http://localhost:5173/api/similarity', json=params)
response.json()

In [ ]:
keys = queries['anchor'].to_list()
responses = []

for k1 in keys:
  pairs = []
  for k2 in keys:
      if k1 != k2:
        pairs.append((k1, k2))
  response = requests.post('http://localhost:5173/api/similarity', json=pairs)
  for r in response.json():
    responses.append(r)

In [ ]:
import requests

response = requests.post('http://localhost:5173/api/data/filter', json={ "filters": ['Characters', 'Novels', 'Oddities', 'Oddity Characters', 'Oddity Inflicted Characters', 'Oddity Specialists', 'Oddity Type', 'Terminology', 'Vampire Hunter', 'Vampires']})

In [ ]:
nodes = response.json()

In [ ]:
nodes[0]
id = nodes[0].get("id")
descr = nodes[0].get("Description")

print(id)
print(descr)

for _ in nodes[0].keys():
    if type(nodes[0][_]) == dict:
        for __ in nodes[0][_].keys():
            if(__ == "text") and len(nodes[0][_][__]) > 0:
                print(f"{_} -> {__}: {nodes[0][_][__]}")

In [ ]:
dataset = []

for node in nodes:
    id = node.get("id")
    descr = node.get("Description")

    dataset.append({
        "anchor": id,
        "positive": descr
    })

    for _ in node.keys():
        if type(node[_]).__name__ == 'dict':
            for __ in node[_].keys():
                if(__ == "text") and len(node[_][__]) > 0:
                    dataset.append({
                        "anchor": id,
                        "positive": node[_][__]
                    })

dataset[0]

In [ ]:
dataset = pd.DataFrame(dataset)
dataset.drop_duplicates(inplace=True)
dataset.reset_index(drop=True, inplace=True)
dataset.describe()

In [ ]:
dataset['anchor'].unique()

In [ ]:
dataset

In [ ]:
import re
def has_html(text):
    return bool(re.search(r'<[^>]+>', text))

def remove_html(text):
    return re.sub(r'<[^>]+>', '', text)


In [ ]:
dataset["positive"] = dataset["positive"].str.replace(r'\(.*?\)', '', regex=True)
dataset["positive"] = dataset["positive"].str.replace(r'\[.*?\]', '', regex=True)
dataset["positive"] = dataset["positive"].str.replace(r'\n', ' ', regex=True)
dataset["positive"] = dataset["positive"].str.replace(r'\s+', ' ', regex=True)

In [ ]:
dataset['positive'] = dataset['positive'].str.strip()
dataset['positive'] = dataset['positive'].str.lower()
dataset

In [ ]:
df = dataset.copy()
df["has_html"] = df["positive"].apply(has_html)
df[df["has_html"] == True]

dataset.drop(df[df["has_html"] == True].index, inplace=True)
dataset.reset_index(drop=True, inplace=True)

In [ ]:
dataset.to_json('---', orient='records')

In [ ]:
dataset = pd.read_json('---')
dataset

In [ ]:
quotes = dataset[dataset['positive'].str.contains(r'^".*"')].copy()
quotes.drop(quotes[quotes['anchor'].str.contains('monogatari')].index, inplace=True)
quotes

In [ ]:
quotes['positive'] = quotes['positive'].str.split(r'"\s"')
quotes = quotes.explode('positive')

In [ ]:
quotes['positive'] = quotes['positive'].str.replace(r'"', '', regex=True)
quotes['positive'] = quotes['positive'].str.strip()
quotes

In [ ]:
no_quotes = dataset.drop(quotes.index)

In [ ]:
# split positive by phrases
no_quotes['positive'] = no_quotes['positive'].str.split(r'\.\s+')
no_quotes = no_quotes.explode('positive')
no_quotes.reset_index(drop=True, inplace=True)
no_quotes

In [ ]:
drop = no_quotes.query('positive.str.len() < 35')
no_quotes.drop(drop.index, inplace=True)

In [ ]:
dataset = pd.concat([no_quotes, quotes])
dataset.reset_index(drop=True, inplace=True)
dataset

In [ ]:
dataset.sort_values(by='anchor', inplace=True)
dataset.drop_duplicates(inplace=True)
dataset.dropna(inplace=True)
dataset.reset_index(drop=True, inplace=True)
dataset

In [ ]:
dataset.to_json('---', orient='records')

In [ ]:
dataset = pd.read_json('---')
dataset

In [ ]:
dataset.sample(10)

In [ ]:
import random

def mutate_text(text, temp=0.4):
  """
  has a chance of mutating the text possible mutations are:
    - removing 'monogatari' from the text if present
    - dropping words if the text has more than 3 words
    - order reversal of words in the text

  Args:
      text (str): 'anchor' column value
      temp (float): temperature value for mutation
  """
  
  # check if the text contains 'monogatari'
  if 'monogatari' in text:
    if(random.random() > temp):
      return text
    text = text.replace('monogatari', '')
    return text.strip()
    
  if(random.random() > temp):
    return text
  
  # check if text has more than 1 word
  words = text.split()
  if len(words) == 2:
    choice = random.choice(["drop", "reverse"])
    if choice == "drop":
      text = random.choice(words)
      return text
    else:
      text = ' '.join(words[::-1])
      return text
  elif len(words) > 2:
    text = random.choice(words)
    
  return text

In [ ]:
import random

def introduce_typo(word):
    typo_type = random.choice(["substitution", "insertion", "deletion", "transposition"])
    idx = random.randint(0, len(word) - 1)

    if typo_type == "substitution":
        word = word[:idx] + random.choice("abcdefghijklmnopqrstuvwxyz") + word[idx+1:]

    elif typo_type == "insertion":
        word = word[:idx] + random.choice("abcdefghijklmnopqrstuvwxyz") + word[idx:]

    elif typo_type == "deletion" and len(word) > 1:
        word = word[:idx] + word[idx+1:]

    elif typo_type == "transposition" and len(word) > 1:
        if idx < len(word) - 1:
            word = word[:idx] + word[idx+1] + word[idx] + word[idx+2:]

    return word

query = "Monogatari series"
for _ in range(5):
    print(introduce_typo(query))

In [ ]:
mutated = dataset.copy()
entries = pd.DataFrame(columns=['anchor', 'positive', 'negative'])

In [ ]:
for row in mutated.iterrows():
    index, data = row
    
    print(data['anchor'])
    try:
        adj = pd.DataFrame(get_adjacent(data['anchor'])).dropna().sample().iloc[0]
        positive = pd.DataFrame(get_positive(adj[0])).dropna().sample().iloc[0].loc['document']
        entries.loc[len(entries)] = [data['anchor'], data['positive'], positive]
    except Exception as e:
        print(f'\tERROR -> {e}')
        continue

entries

In [ ]:
entries.to_dict(orient='records')

In [ ]:
split = entries.copy()
split['negative'] = split['negative'].apply(lambda x: x.split(". ")[0])
split

In [ ]:
split.to_dict(orient='records')

In [ ]:
mutated = split.copy()

In [ ]:
mutated['anchor'] = dataset['anchor'].apply(lambda x: mutate_text(x,.6))

In [ ]:
mutated.dropna(inplace=True)
mutated['anchor'].unique().tolist()

In [ ]:
drop = [...]

mutated.drop(mutated[mutated['anchor'].isin(drop)].index, inplace=True)
mutated.reset_index(drop=True, inplace=True)
mutated

In [ ]:
mutated[mutated['anchor'].str.contains(r"'s$")]

In [ ]:
mutated.drop(mutated[mutated['anchor'].str.contains(r"'s$")].index, inplace=True)
mutated.drop(mutated[mutated['positive'].str.contains(r'[ぁ-んァ-ン一-龯]+')].index, inplace=True)
mutated.reset_index(drop=True, inplace=True)

In [ ]:
mutated.to_json('---', orient='records')

In [ ]:
df['anchor'].sample(5).to_list()

In [ ]:
for _ in df['anchor'].sample(5).to_list():
    print(f'{_} -> {mutate_text(_)}')

In [ ]:
df = dataset.copy()
df['anchor'] = df['anchor'].str.replace(r'-', '', regex=True)
df['anchor'] = df['anchor'].str.replace(r'\'s', '', regex=True)

In [ ]:
spaces = df['anchor'].str.contains(r'\s')
spaces = df[spaces]
spaces['anchor'].unique()

In [ ]:
# split the anchor into words
spaces.loc[:, 'anchor'] = spaces['anchor'].str.split(' ')

In [ ]:
exploded = spaces.explode('anchor').reset_index(drop=True)
exploded

In [ ]:
inverted = df['anchor'].str.contains(r'\s')
inverted = df[inverted]

# split the anchor into words and reverse the order
inverted.loc[:, 'anchor'] = inverted['anchor'].str.split(' ')
inverted.loc[:, 'anchor'] = inverted['anchor'].apply(lambda x: x[::-1])
inverted.loc[:, 'anchor'] = inverted['anchor'].apply(lambda x: ' '.join(x))
inverted

In [ ]:
mono = df.query('anchor.str.contains("monogatari") == True')
mono['anchor'].unique()

In [ ]:
mono.loc[:, 'anchor'] = mono['anchor'].str.replace(r'monogatari', '', regex=True)
mono.loc[:, 'anchor'] = mono['anchor'].str.strip()
mono

In [ ]:
merge = pd.concat([exploded, inverted, mono, dataset])
merge

In [ ]:
merge.drop_duplicates(inplace=True)
merge.reset_index(drop=True, inplace=True)
merge

In [ ]:
# find positive phrases that contain japanese characters
japanese = merge[merge['positive'].str.contains(r'[ぁ-んァ-ン一-龯]+')]
merge.drop(japanese.index, inplace=True)
merge.reset_index(drop=True, inplace=True)
merge

In [ ]:
disambiguations = requests.post('http://localhost:5173/api/data/filter', json={"filters": ["disambiguations"]}).json()

In [ ]:
remove = []
for dis in disambiguations:
    remove.append(dis.get("id"))
    
remove

In [ ]:
# remove disambiguations
merge.drop(merge[merge['anchor'].isin(remove)].index, inplace=True)
merge.reset_index(drop=True, inplace=True)
merge

In [ ]:
merge['anchor'].sort_values(key=lambda x: x.str.len()).unique()

In [ ]:
drop = [...]

merge.drop(merge[merge['anchor'].isin(drop)].index, inplace=True)
merge.reset_index(drop=True, inplace=True)
merge

In [ ]:
merge.duplicated(subset=['positive']).sum()

In [ ]:
merge.drop_duplicates(subset=['positive'], inplace=True)

In [ ]:
merge['anchor'].sort_values(key=lambda x: x.str.len()).unique()

In [ ]:
merge.to_json('---', orient='records')